Accuracy is the top priority for this project. so xgboost is used as the model. 

In [ ]:
# import xgboost libraries
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import shapely

In [3]:
# Make sure your data_cleaning.py is in the same folder or in your PYTHONPATH
from dataCleaning import load_data, clean_data

df = load_data("climate_change_impact_on_agriculture_2024.csv")
df_clean = clean_data(df)

In [9]:
# Load your data
X = df_clean.drop("Crop_Yield_MT_per_HA", axis=1)  # Replace with your target column name
y = df_clean["Crop_Yield_MT_per_HA"]

# HOT ENCODE categorical variables
X = pd.get_dummies(X, drop_first=True)

In [12]:
# Step 1: Split into train+val and test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Split train+val into train and validation
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)
# (0.25 * 0.8 = 0.2 → 60% train, 20% val, 20% test)

# Initialize XGBoost model
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    eval_metric='rmse',
    random_state=42
)

# Train model with validation
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False  # Set to True if you want to see training log
)

# Evaluate on test set
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Test RMSE: {rmse:.4f}")

# Calculate R-squared
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)
print(f"Test R-squared: {r2:.4f}")


Test RMSE: 0.6626
Test R-squared: 0.5840


In [ ]:
# Tune the model using GridSearchCV
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0]
}
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1  # Use all available cores
)
# Fit the grid search
grid_search.fit(X_train, y_train)
# Get the best parameters and score
best_params = grid_search.best_params_
best_score = -grid_search.best_score_  # Negate because we used neg_mean_squared_error
print(f"Best parameters: {best_params}")
print(f"Best cross-validated RMSE: {best_score:.4f}")
# Train the model with the best parameters
model_best = XGBRegressor(
    **best_params,
    objective='reg:squarederror',
    eval_metric='rmse',
    random_state=42
)
model_best.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)
# Evaluate the best model on the test set
y_pred_best = model_best.predict(X_test)
rmse_best = mean_squared_error(y_test, y_pred_best, squared=False)
print(f"Test RMSE with best parameters: {rmse_best:.4f}")
# Calculate R-squared for the best model
r2_best = r2_score(y_test, y_pred_best)
print(f"Test R-squared with best parameters: {r2_best:.4f}")


Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best parameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Best cross-validated RMSE: 0.3851
Test RMSE with best parameters: 0.6161
Test R-squared with best parameters: 0.6404


In [17]:
# use shapely to calculate feature importance
import shap
# Create a SHAP explainer
explainer = shap.Explainer(model_best, X_train)
# Calculate SHAP values
shap_values = explainer(X_test)
# Plot SHAP summary
shap.summary_plot(shap_values, X_test, plot_type="bar", max_display=10)
# Save the model
model_best.save_model("xgboost_model.json")
# Save the SHAP values
shap_values_df = pd.DataFrame(shap_values.values, columns=X_test.columns)
shap_values_df.to_csv("shap_values.csv", index=False)
# Save the model and SHAP values        

ModuleNotFoundError: No module named 'shap'